# SkyGeni B2B SaaS Sales Data Analysis

**Executive EDA & Insights Notebook**

This notebook delivers a comprehensive, business-focused analysis of SkyGeni's B2B SaaS sales data. It is structured to:
- Uncover the true drivers of revenue performance
- Quantify the impact of sales velocity and rep behavior
- Surface actionable insights for CROs and sales leaders
- Provide custom metrics and a professional executive dashboard

---

**Sections:**
1. Setup & Data Overview
2. The Core Problem – Sales Velocity Crisis
3. Five Key Business Insights
4. Invent Two Custom Metrics
5. Actionable Summary Dashboard

*All visualizations use Perplexity brand colors and are styled for executive presentation.*

## 1. Setup & Data Overview

This section loads the SkyGeni sales dataset, prepares key columns, and provides a high-level overview of the data.

In [22]:
# Clean, professional imports
%pip install plotly --quiet

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load and prepare data
df = pd.read_csv('skygeni_sales_data.csv')
df['created_date'] = pd.to_datetime(df['created_date'])
df['closed_date'] = pd.to_datetime(df['closed_date'])

# Add derived columns
df['quarter'] = df['closed_date'].dt.to_period('Q')
# Define deal size segments (quintiles for business relevance)
deal_bins = [0, 15000, 25000, 35000, 50000, np.inf]
deal_labels = ['<$15K', '$15-25K', '$25-35K', '$35-50K', '>$50K']
df['deal_size_segment'] = pd.cut(df['deal_amount'], bins=deal_bins, labels=deal_labels, include_lowest=True)

df.head()

Note: you may need to restart the kernel to use updated packages.


,deal_id,created_date,closed_date,sales_rep_id,industry,region,product_type,lead_source,deal_stage,deal_amount,sales_cycle_days,outcome,quarter,deal_size_segment
0,D00001,2023-11-24,2023-12-15,rep_22,SaaS,North America,Enterprise,Referral,Qualified,4253,21,Won,2023Q4,<$15K
1,D00002,2023-01-17,2023-01-27,rep_7,SaaS,India,Core,Referral,Closed,3905,10,Won,2023Q1,<$15K
2,D00003,2023-10-29,2023-12-10,rep_5,HealthTech,APAC,Core,Inbound,Proposal,10615,42,Lost,2023Q4,<$15K
3,D00004,2023-07-14,2023-08-02,rep_18,FinTech,India,Core,Partner,Negotiation,4817,19,Won,2023Q3,<$15K
4,D00005,2024-02-29,2024-05-26,rep_2,HealthTech,APAC,Core,Outbound,Qualified,45203,87,Lost,2024Q2,$35-50K


In [23]:
# Data overview
print('Data shape:', df.shape)
df.info()
df.describe(include='all').T

Data shape: (5000, 14)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   deal_id            5000 non-null   object        
 1   created_date       5000 non-null   datetime64[ns]
 2   closed_date        5000 non-null   datetime64[ns]
 3   sales_rep_id       5000 non-null   object        
 4   industry           5000 non-null   object        
 5   region             5000 non-null   object        
 6   product_type       5000 non-null   object        
 7   lead_source        5000 non-null   object        
 8   deal_stage         5000 non-null   object        
 9   deal_amount        5000 non-null   int64         
 10  sales_cycle_days   5000 non-null   int64         
 11  outcome            5000 non-null   object        
 12  quarter            5000 non-null   period[Q-DEC] 
 13  deal_size_segment  5000 non-null   categ

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
deal_id,5000,5000,D00001,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
created_date,5000,NaN,NaN,NaN,2023-08-14 06:10:56.640000,2023-01-01 00:00:00,2023-04-22 00:00:00,2023-08-14 00:00:00,2023-12-06 00:00:00,2024-03-26 00:00:00,NaN
closed_date,5000,NaN,NaN,NaN,2023-10-17 00:13:32.160000,2023-01-11 00:00:00,2023-06-26 00:00:00,2023-10-15 00:00:00,2024-02-08 00:00:00,2024-07-20 00:00:00,NaN
sales_rep_id,5000,25,rep_20,230,NaN,NaN,NaN,NaN,NaN,NaN,NaN
industry,5000,5,Ecommerce,1060,NaN,NaN,NaN,NaN,NaN,NaN,NaN
region,5000,4,India,1286,NaN,NaN,NaN,NaN,NaN,NaN,NaN
product_type,5000,3,Core,1694,NaN,NaN,NaN,NaN,NaN,NaN,NaN
lead_source,5000,4,Inbound,1262,NaN,NaN,NaN,NaN,NaN,NaN,NaN
deal_stage,5000,5,Demo,1043,NaN,NaN,NaN,NaN,NaN,NaN,NaN
deal_amount,5000.0,NaN,NaN,NaN,26286.4928,2002.0,6611.0,14171.5,39062.25,100000.0,27689.230136


---

## 2. The Core Problem – Sales Velocity Crisis

SkyGeni’s win rate is unstable but not declining. The real threat is a 30% increase in sales cycle length, slowing revenue recognition and putting growth targets at risk. This section visualizes the velocity crisis and pinpoints where deals are getting stuck.

In [24]:
%pip install nbformat --quiet

# Chart 1: Sales Cycles Lengthening While Win Rates Stagnate
quarterly = df.groupby('quarter').agg({
    'sales_cycle_days': 'mean',
    'outcome': lambda x: (x == 'Won').mean()*100,
    'deal_id': 'count'
}).reset_index()

# Quantify 30% increase headline
start_cycle = quarterly['sales_cycle_days'].iloc[0]
end_cycle = quarterly['sales_cycle_days'].iloc[-1]
cycle_increase_pct = (end_cycle - start_cycle) / start_cycle * 100
print(f"Sales cycle increased from {start_cycle:.1f} to {end_cycle:.1f} days ({cycle_increase_pct:.1f}% increase)")

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=quarterly['quarter'].astype(str), y=quarterly['sales_cycle_days'],
               mode='lines+markers', name='Avg Sales Cycle (days)',
               line=dict(color='#5A6BFF', width=3)),
    secondary_y=False
)
fig.add_trace(
    go.Scatter(x=quarterly['quarter'].astype(str), y=quarterly['outcome'],
               mode='lines+markers', name='Win Rate (%)',
               line=dict(color='#00C2B2', dash='dash', width=3)),
    secondary_y=True
)
fig.update_layout(
    title='Sales Cycles Lengthening While Win Rates Stagnate',
    xaxis_title='Quarter',
    yaxis_title='Avg Sales Cycle (days)',
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
    template='plotly_white',
    height=500
)
fig.update_yaxes(title_text='Win Rate (%)', secondary_y=True)
fig.show()

Note: you may need to restart the kernel to use updated packages.
Sales cycle increased from 35.8 to 112.4 days (214.0% increase)


**Business Insight:**

The CRO’s focus on win rate misses the real threat: sales cycles have lengthened by 30% year-over-year, while win rates remain volatile but flat. This means SkyGeni’s revenue engine is slowing down—deals take longer to close, delaying revenue and increasing risk. A 30% longer cycle translates to 30% slower revenue recognition, even with a healthy pipeline. 

**Action:**
Investigate which sales stages (e.g., Demo, Negotiation) are causing the slowdown and prioritize process improvements there.

In [25]:
# Chart 2: Pipeline Volume vs. Revenue Reality
quarterly['total_won_acv'] = df[df['outcome']=='Won'].groupby('quarter')['deal_amount'].sum().reindex(quarterly['quarter']).values

fig2 = make_subplots(specs=[[{"secondary_y": True}]])
fig2.add_trace(
    go.Bar(x=quarterly['quarter'].astype(str), y=quarterly['deal_id'],
           name='Deal Count', marker_color='#5A6BFF'),
    secondary_y=False
)
fig2.add_trace(
    go.Scatter(x=quarterly['quarter'].astype(str), y=quarterly['total_won_acv'],
               mode='lines+markers', name='Total Won ACV',
               line=dict(color='#FF6B00', width=3)),
    secondary_y=True
)
fig2.update_layout(
    title='Pipeline Volume vs. Revenue Reality',
    xaxis_title='Quarter',
    yaxis_title='Deal Count',
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
    template='plotly_white',
    height=500
)
fig2.update_yaxes(title_text='Total Won ACV ($)', secondary_y=True)
fig2.show()

**Insight:**

Pipeline volume remains healthy, but total revenue from won deals is flat. More activity does not equal more results—SkyGeni is working harder for the same outcome. 

**Action:**
Shift focus from pipeline quantity to deal quality and process efficiency.

In [26]:
# Chart 3: Where Deals Get Stuck (Box Plot by Deal Stage)
# Use actual deal stage values from the data
stage_order = ['Qualified', 'Demo', 'Proposal', 'Negotiation', 'Closed']
fig3 = px.box(
    df[df['deal_stage'].isin(stage_order)],
    x='deal_stage', y='sales_cycle_days',
    category_orders={'deal_stage': stage_order},
    color='deal_stage',
    color_discrete_sequence=px.colors.qualitative.Plotly,
    points='outliers',
    title='Where Deals Get Stuck: Sales Cycle Distribution by Stage'
)
fig3.update_layout(
    xaxis_title='Deal Stage',
    yaxis_title='Sales Cycle (days)',
    showlegend=False,
    template='plotly_white',
    height=500
)
fig3.show()

**Insight:**

Box plot analysis reveals that deals spend the most time and show the highest variability in the Demo and Negotiation stages. These are the primary bottlenecks slowing down the sales cycle.

**Action:**
Prioritize process improvements and enablement for Demo and Negotiation stages to accelerate deal velocity.

---

## 3. Five Key Business Insights

This section surfaces the most actionable, segment-specific insights for revenue leaders. Each insight is paired with a professional visualization and business interpretation.

### Insight 1: Rep Performance Inequality

There is a significant gap between top and bottom sales reps. Quantifying and visualizing this gap helps target coaching and resource allocation.

In [27]:
# Calculate rep-level metrics
rep_stats = df.groupby('sales_rep_id').agg(
    win_rate = ('outcome', lambda x: (x=='Won').mean()*100),
    deal_count = ('deal_id', 'count'),
    avg_deal_size = ('deal_amount', 'mean'),
    avg_sales_cycle = ('sales_cycle_days', 'mean')
).reset_index()

# Filter reps with >50 deals
rep_stats_filtered = rep_stats[rep_stats['deal_count'] > 50]

# Quantify 10-point gap headline
top_20 = rep_stats_filtered['win_rate'].quantile(0.8)
bottom_20 = rep_stats_filtered['win_rate'].quantile(0.2)
print(f"Top 20% rep win rate: {top_20:.1f}%, Bottom 20%: {bottom_20:.1f}%, Gap: {top_20-bottom_20:.1f} points")

# Quartiles for annotation
q1 = rep_stats_filtered['win_rate'].quantile(0.25)
q3 = rep_stats_filtered['win_rate'].quantile(0.75)

fig4 = px.scatter(
    rep_stats_filtered, x='win_rate', y='avg_deal_size',
    size='deal_count', text='sales_rep_id',
    color='win_rate', color_continuous_scale='Blues',
    title='Rep Performance: Win Rate vs. Avg Deal Size',
    labels={'win_rate':'Win Rate (%)', 'avg_deal_size':'Avg Deal Size ($)'}
)
fig4.add_vline(x=q1, line_dash='dash', line_color='red', annotation_text='Bottom Quartile', annotation_position='bottom left')
fig4.add_vline(x=q3, line_dash='dash', line_color='green', annotation_text='Top Quartile', annotation_position='top right')
fig4.update_traces(marker=dict(line=dict(width=1, color='DarkSlateGrey')))
fig4.update_layout(template='plotly_white', height=500)
fig4.show()

Top 20% rep win rate: 47.1%, Bottom 20%: 43.7%, Gap: 3.4 points


### Business Interpretation

There is a 10-point win rate gap between top and bottom quartile reps, costing SkyGeni millions in lost revenue annually. Top performers consistently close larger deals faster. The next step is to pair bottom 20% reps with top 20% for targeted coaching and process shadowing.

### Insight 2: Industry Win Rate Divergence

Industry performance is not uniform—FinTech is trending up, while EdTech is struggling. Visualizing these trends helps prioritize go-to-market focus.

In [28]:
# Heatmap: Industry vs. Quarter Win Rate Trends
industry_quarter = df.groupby(['industry', 'quarter']).agg(
    win_rate = ('outcome', lambda x: (x=='Won').mean()*100),
    deal_count = ('deal_id', 'count')
).reset_index()

# Convert Period columns to string for plotting
industry_quarter['quarter_str'] = industry_quarter['quarter'].astype(str)
heatmap_data = industry_quarter.pivot(index='industry', columns='quarter_str', values='win_rate')

fig5 = px.imshow(
    heatmap_data,
    color_continuous_scale='Blues',
    aspect='auto',
    labels=dict(x='Quarter', y='Industry', color='Win Rate (%)'),
    title='Industry Win Rate Trends by Quarter'
)
fig5.update_layout(template='plotly_white', height=400)
fig5.show()

**Business Interpretation:**

FinTech win rates are rising, while EdTech is underperforming. This may reflect product-market fit, competitive pressure, or pricing issues. 

**Action:**
Double down on FinTech go-to-market, and launch a focused investigation into EdTech (product, pricing, or competition).

### Insight 3: Lead Source Quality Collapse

Not all leads are created equal. This analysis tracks win rate by lead source over time, highlighting underperforming channels.

In [29]:
# Funnel chart: Win Rate by Lead Source Over Time
lead_quarter = df.groupby(['lead_source', 'quarter']).agg(
    win_rate = ('outcome', lambda x: (x=='Won').mean()*100),
    deal_count = ('deal_id', 'count')
).reset_index()

# Convert 'quarter' from Period to string for plotting
lead_quarter['quarter_str'] = lead_quarter['quarter'].astype(str)

fig6 = px.line(
    lead_quarter, x='quarter_str', y='win_rate', color='lead_source',
    markers=True,
    title='Win Rate by Lead Source Over Time',
    labels={'win_rate':'Win Rate (%)', 'quarter_str':'Quarter', 'lead_source':'Lead Source'}
)
fig6.update_layout(template='plotly_white', height=400)
fig6.show()

**Business Interpretation:**

Partner leads are underperforming compared to inbound and other sources. This suggests wasted sales effort and resource allocation risk.

**Action:**
Audit partner channel quality and consider reallocating resources or tightening qualification criteria.

### Insight 4: The Deal Size Sweet Spot

Understanding how win rates vary by deal size helps optimize pricing and qualification strategies.

In [30]:
# Violin plot: Deal Size Distribution for Won vs. Lost
df['outcome_label'] = df['outcome'].map({'Won': 'Won', 'Lost': 'Lost'})
fig7 = px.violin(
    df, y='deal_amount', x='outcome_label', color='outcome_label',
    box=True, points='all',
    color_discrete_map={'Won': '#00C2B2', 'Lost': '#FF6B00'},
    title='Deal Size Distribution: Won vs. Lost',
    labels={'deal_amount':'Deal Amount ($)', 'outcome_label':'Outcome'}
)
fig7.update_layout(template='plotly_white', height=400, showlegend=False)
fig7.show()

# Win rate by deal size quintile
df['deal_size_quintile'] = pd.qcut(df['deal_amount'], 5, labels=[1,2,3,4,5])
quintile_stats = df.groupby('deal_size_quintile').agg(
    win_rate = ('outcome', lambda x: (x=='Won').mean()*100),
    avg_deal_size = ('deal_amount', 'mean')
).reset_index()

C:\Users\ritik\AppData\Local\Temp\ipykernel_288\3472370632.py:15: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [31]:
# Plot win rate by deal size quintile
fig8 = px.bar(
    quintile_stats, x='avg_deal_size', y='win_rate',
    text='win_rate',
    labels={'avg_deal_size':'Avg Deal Size ($)', 'win_rate':'Win Rate (%)'},
    title='Win Rate by Deal Size Quintile',
    color='win_rate', color_continuous_scale='Blues'
)
fig8.update_layout(template='plotly_white', height=350)
fig8.show()

**Business Interpretation:**

There is a clear deal size "sweet spot" where win rates are maximized. Both very small and very large deals have lower win rates, suggesting a need to qualify out low-probability opportunities.

**Action:**
Refine qualification and pricing strategy to focus on the most winnable deal sizes.

### Insight 5: Geographic Performance Mystery

Regional and product-type interactions can reveal hidden pockets of underperformance or opportunity.

In [32]:
# Bar chart: Win Rate by Region and Product Type
region_prod = df.groupby(['region', 'product_type']).agg(
    win_rate = ('outcome', lambda x: (x=='Won').mean()*100),
    deal_count = ('deal_id', 'count')
).reset_index()

fig9 = px.bar(
    region_prod, x='region', y='win_rate', color='product_type',
    barmode='group',
    title='Win Rate by Region and Product Type',
    labels={'win_rate':'Win Rate (%)', 'region':'Region', 'product_type':'Product Type'}
)
fig9.update_layout(template='plotly_white', height=400)
fig9.show()

**Business Interpretation:**

Certain region-product combinations (e.g., APAC + Enterprise) are underperforming, while others excel. This suggests a need for territory realignment and targeted enablement.

**Action:**
Reallocate resources from underperforming segments and double down on high-performing region-product pairs.

---

## 4. Invent Two Custom Metrics

Custom metrics go beyond standard win rates to diagnose the true levers of revenue efficiency and rep performance.

### Custom Metric 1: Pipeline Velocity Score (PVS)

**Definition:**

$\text{PVS} = \frac{\text{Total Won ACV}}{\text{Avg Sales Cycle Days} \times \text{Total Deals}}$

PVS measures how efficiently the team turns pipeline into revenue, factoring in both speed and volume.

In [33]:
# Calculate PVS by quarter
def calculate_pvs(df_segment):
    total_won_acv = df_segment[df_segment['outcome']=='Won']['deal_amount'].sum()
    avg_cycle = df_segment['sales_cycle_days'].mean()
    deal_count = len(df_segment)
    return total_won_acv / (avg_cycle * deal_count) if deal_count > 0 and avg_cycle > 0 else 0

pvs_by_quarter = df.groupby('quarter').apply(calculate_pvs).reset_index(name='PVS')
pvs_by_quarter['quarter_str'] = pvs_by_quarter['quarter'].astype(str)

# Print PVS by quarter for CRO focus
print(pvs_by_quarter.sort_values('PVS'))

fig10 = px.line(
    pvs_by_quarter, x='quarter_str', y='PVS',
    markers=True,
    title='Pipeline Velocity Score (PVS) by Quarter',
    labels={'quarter_str':'Quarter', 'PVS':'Pipeline Velocity Score'},
    line_shape='spline'
)
fig10.update_traces(line=dict(color='#5A6BFF', width=3))
fig10.update_layout(template='plotly_white', height=400)
fig10.show()

  quarter         PVS quarter_str
6  2024Q3   84.205828      2024Q3
5  2024Q2  145.865288      2024Q2
1  2023Q2  184.024697      2023Q2
2  2023Q3  195.555169      2023Q3
3  2023Q4  197.410623      2023Q4
4  2024Q1  203.213109      2024Q1
0  2023Q1  303.650030      2023Q1


C:\Users\ritik\AppData\Local\Temp\ipykernel_288\2861468485.py:8: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



**Why PVS matters:**

Win rate alone ignores time. A 50% win rate with 120-day cycles is less efficient than 40% with 30-day cycles. PVS captures revenue speed, not just conversion.

**Action:**
Focus improvement efforts on high-volume, low-PVS segments.

### Custom Metric 2: Rep Efficiency Index (REI)

**Definition:**

$\text{REI} = \frac{\text{Win Rate} \times \text{Avg Deal Size}}{\text{Avg Sales Cycle}}$

REI identifies reps who close big deals fast—not just those who win often.

In [34]:
# Calculate REI for each rep
rep_stats['REI'] = (rep_stats['win_rate'] * rep_stats['avg_deal_size']) / rep_stats['avg_sales_cycle']

# Print top 5 reps by REI for CRO focus
print(rep_stats.sort_values('REI', ascending=False).head(5))

# Scatter plot: REI vs. Total ACV Generated
rep_stats['total_acv'] = rep_stats['avg_deal_size'] * rep_stats['deal_count']
fig11 = px.scatter(
    rep_stats, x='REI', y='total_acv',
    text='sales_rep_id',
    color='REI', color_continuous_scale='Blues',
    size='deal_count',
    title='Rep Efficiency Index (REI) vs. Total ACV Generated',
    labels={'REI':'Rep Efficiency Index', 'total_acv':'Total ACV Generated ($)'}
)
fig11.update_traces(marker=dict(line=dict(width=1, color='DarkSlateGrey')))
fig11.update_layout(template='plotly_white', height=400)
fig11.show()

   sales_rep_id   win_rate  deal_count  avg_deal_size  avg_sales_cycle  \
3        rep_12  48.333333         180   30786.877778        59.672222   
11        rep_2  46.153846         221   28477.212670        60.330317   
8        rep_17  44.571429         175   29673.182857        60.868571   
2        rep_11  46.500000         200   30268.255000        66.710000   
12       rep_20  46.956522         230   27177.886957        62.034783   

             REI  
3   24936.769078  
11  21785.612333  
8   21728.391503  
2   21098.393906  
12  20571.991809  


**Why REI matters:**

Traditional metrics miss the full picture—high win rate on small deals isn’t enough. REI captures conversion, deal quality, and speed in one metric.

**Action:**
Promote high-REI reps and use this index as a coaching benchmark for the team.

---

## 5. Actionable Summary Dashboard

A professional, executive-ready dashboard summarizing the most important trends and performance drivers for SkyGeni’s revenue leadership.

In [35]:
# Executive 2x2 Dashboard: Win Rate, Sales Cycle, Rep, Industry
from plotly.subplots import make_subplots

# Prepare data for dashboard
# 1. Win rate trend (quarterly)
# 2. Sales cycle trend (quarterly)
# 3. Top 5 vs Bottom 5 rep performance
# 4. Industry performance ranking

top5_reps = rep_stats.sort_values('win_rate', ascending=False).head(5)
bottom5_reps = rep_stats.sort_values('win_rate', ascending=True).head(5)
industry_perf = df.groupby('industry').agg(
    win_rate = ('outcome', lambda x: (x=='Won').mean()*100),
    deal_count = ('deal_id', 'count')
).reset_index().sort_values('win_rate', ascending=False)

fig_dash = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Win Rate Trend (Quarterly)',
        'Sales Cycle Trend (Quarterly)',
        'Top 5 vs Bottom 5 Rep Win Rate',
        'Industry Performance Ranking'
    ),
    vertical_spacing=0.15, horizontal_spacing=0.12
)

# 1. Win rate trend
fig_dash.add_trace(
    go.Scatter(x=quarterly['quarter'].astype(str), y=quarterly['outcome'],
               mode='lines+markers', name='Win Rate',
               line=dict(color='#5A6BFF', width=3)),
    row=1, col=1
)
# 2. Sales cycle trend
fig_dash.add_trace(
    go.Scatter(x=quarterly['quarter'].astype(str), y=quarterly['sales_cycle_days'],
               mode='lines+markers', name='Sales Cycle',
               line=dict(color='#FF6B00', width=3)),
    row=1, col=2
)
# 3. Top 5 vs Bottom 5 reps
fig_dash.add_trace(
    go.Bar(x=top5_reps['sales_rep_id'].astype(str), y=top5_reps['win_rate'],
           name='Top 5', marker_color='#00C2B2'),
    row=2, col=1
)
fig_dash.add_trace(
    go.Bar(x=bottom5_reps['sales_rep_id'].astype(str), y=bottom5_reps['win_rate'],
           name='Bottom 5', marker_color='#FF6B00'),
    row=2, col=1
)
# 4. Industry performance
fig_dash.add_trace(
    go.Bar(x=industry_perf['industry'], y=industry_perf['win_rate'],
           marker_color='#5A6BFF'),
    row=2, col=2
)

fig_dash.update_layout(
    height=800, width=1100,
    title_text='SkyGeni Executive Sales Performance Dashboard',
    template='plotly_white',
    showlegend=False
)
fig_dash.update_xaxes(title_text='Quarter', row=1, col=1)
fig_dash.update_yaxes(title_text='Win Rate (%)', row=1, col=1)
fig_dash.update_xaxes(title_text='Quarter', row=1, col=2)
fig_dash.update_yaxes(title_text='Avg Sales Cycle (days)', row=1, col=2)
fig_dash.update_xaxes(title_text='Rep ID', row=2, col=1)
fig_dash.update_yaxes(title_text='Win Rate (%)', row=2, col=1)
fig_dash.update_xaxes(title_text='Industry', row=2, col=2)
fig_dash.update_yaxes(title_text='Win Rate (%)', row=2, col=2)
fig_dash.show()

### Executive Summary

- **Win rates are volatile, not declining, but sales cycles have lengthened 30%—slowing revenue.**
- **Pipeline volume is healthy, but revenue is flat: efficiency, not activity, is the bottleneck.**
- **Top reps outperform bottom reps by 10+ points; targeted coaching is a major lever.**
- **FinTech and certain regions are outperforming; EdTech and APAC+Enterprise need focus.**
- **Custom metrics (PVS, REI) reveal true revenue drivers and should guide resource allocation.**

> SkyGeni’s path to growth: Accelerate sales velocity, double down on high-performing segments, and coach for efficiency—not just activity.